<a href="https://colab.research.google.com/github/eunicedai/Integrative-Analyze/blob/master/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Homework 3** 

> TaiHsuan Wu | 2524355409


In [0]:
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
!pip install -q pyomo

In [0]:
from __future__ import division
from pyomo.environ import *

In [0]:
!apt-get install -y -qq glpk-utils

1) Formulate & solve Two “All-in-One” LPs (as in class).
For the first model, assume that the plan includes
specifying how many type 1 orders are used for

> $min\ \ c_1x_1+c_2x_2-r_1(z_{11})-r_2(z_{12}+z_{22})+0.2(x_1-z_{11}-z_{12}+x_2-z_{22})\\+P_H(10(lack1_H+lack2_H)+0.2(more1_H+more2_H))\\+P_L(10(lack1_L+lack2_L)+0.2(more1_L+more2_L))$

> $s.t\ \ x_1-z_{11}-z_{12}\ge 0\\x_2-z_{22}\ge 0\\ z_{11}+lack_{1H}+more_{1H} = d_{1H}\\ z_{11}+lack_{1L}+more_{1L} = d_{1L}\\ z_{12}+z_{22}+lack_{2H}+more_{2H}=d_{2H}\\ z_{12}+z_{22}+lack_{2L}+more_{2L}=d_{2L}$

In [0]:
def model1():
  m = ConcreteModel()

  m.x1 = Var(within=NonNegativeReals)
  m.x2 = Var(within=NonNegativeReals)

  m.lack1 = Var(range(2), within=NonNegativeReals)
  m.lack2 = Var(range(2), within=NonNegativeReals)
  m.more1 = Var(range(2), within=NonNegativeReals)
  m.more2 = Var(range(2), within=NonNegativeReals)
  m.z11 = Var(within=NonNegativeReals)
  m.z12 = Var(within=NonNegativeReals)
  m.z22 = Var(within=NonNegativeReals)

  m.cost = Expression(initialize=(4*m.x1+2*m.x2))
  m.rev = Expression(initialize=(-8*m.z11-4*(m.z12 + m.z22)+0.2*(m.x1-m.z11-m.z12+m.x2-m.z22)))
  m.high = Expression(initialize=(10*(m.lack1[0]+m.lack2[0])+0.2*(m.more1[0]+m.more2[0])))
  m.low = Expression(initialize=(10*(m.lack1[1]+m.lack2[1])+0.2*(m.more1[1]+m.more2[1])))

  m.obj = Objective(expr= m.cost+m.rev+0.3*m.high+0.7*m.low, sense=minimize)

  m.limit = ConstraintList()
  m.limit.add(expr= m.x1-m.z12-m.z11 >= 0)
  m.limit.add(expr= m.x2-m.z22 >= 0)
  m.limit.add(expr= m.z11+m.lack1[0]+m.more1[0] == 125)
  m.limit.add(expr= m.z11+m.lack1[1]+m.more1[1] == 75)
  m.limit.add(expr= m.z12+m.z22+m.lack2[0]+m.more2[0] == 100) 
  m.limit.add(expr= m.z12+m.z22+m.lack2[1]+m.more2[1] == 112.5)
  
  SolverFactory('glpk', executable='/usr/bin/glpsol').solve(m)
  return m

In [121]:
model1 = tt()
m1 = pd.DataFrame(columns=['x1', 'x2', 'z11', 'z12', 'z22', 'objective'], index=['value']) 
m1['x1'] = model1.x1()
m1['x2'] = model1.x2()
m1['z11'] = model1.z11()
m1['z12'] = model1.z12()
m1['z22'] = model1.z22()
m1['objective'] = -model1.obj()
m1

,x1,x2,z11,z12,z22,objective
value,75.0,100.0,75.0,0.0,100.0,495.25


2) For the second model, any left over quantity of from stage 1
can be used in stage 2. (Both
are equally weighted)

> $min\ \ c_1x_1+c_2x_2\\+P_H(-r_1z_{11H}-r_2(z_{12H}+z_{22H})+10(lack_{1H}+lack_{2H})+0.2(more_{1H}+more_{2H}))\\+P_L(-r_1z_{11L}-r_2(z_{12L}+z_{22L})+10(lack_{1L}+lack_{2L})+0.2(more_{1L}+more_{2L}))$

> $s.t\ \ z_{11H}+z_{12H}+more_{1H}=x_1\\ z_{11L}+z_{12L}+more_{1L}=x_1\\ z_{22H}+more_{2H}=x_2\\ z_{22L}+more_{2L}=x_2\\ z_{11}+lack_{1H}+more_{1H} = d_{1H}\\ z_{11}+lack_{1L}+more_{1L} = d_{1L}\\ z_{12}+z_{22}+lack_{2H}+more_{2H}=d_{2H}\\ z_{12}+z_{22}+lack_{2L}+more_{2L}=d_{2L}$

In [0]:
def model2():
  m = ConcreteModel()

  m.x1 = Var(within=NonNegativeReals) 
  m.x2 = Var(within=NonNegativeReals) 

  m.lack1 = Var(range(2), within=NonNegativeReals)
  m.lack2 = Var(range(2), within=NonNegativeReals)
  m.more1 = Var(range(2), within=NonNegativeReals)
  m.more2 = Var(range(2), within=NonNegativeReals)
  m.z11 = Var(range(2), within=NonNegativeReals)
  m.z12 = Var(range(2), within=NonNegativeReals)
  m.z22 = Var(range(2), within=NonNegativeReals)
  
  m.cost = Expression(initialize=(-4*m.x1-2*m.x2))
  m.high = Expression(initialize=(8*m.z11[0]+4*(m.z12[0]+m.z22[0])-10*(m.lack1[0]+m.lack2[0])-0.2*(m.more1[0]+m.more2[0]))) 
  m.low = Expression(initialize=(8*m.z11[1]+4*(m.z12[1]+m.z22[1])-10*(m.lack1[1]+m.lack2[1])-0.2*(m.more1[1]+m.more2[1]))) 

  m.obj = Objective(expr = -m.cost-0.3*m.high-0.7*m.low, sense=minimize)

  m.limit = ConstraintList()
  m.limit.add(expr= m.z11[0]+m.z12[0]+m.more1[0] == m.x1) 
  m.limit.add(expr= m.z11[1]+m.z12[1]+m.more1[1] == m.x1) 
  m.limit.add(expr= m.z22[0]+m.more2[0] == m.x2)
  m.limit.add(expr= m.z22[1]+m.more2[1] == m.x2)
  m.limit.add(expr= m.z11[0]+m.lack1[0] == 125)
  m.limit.add(expr= m.z11[1]+m.lack1[1] == 75)
  m.limit.add(expr= m.z12[0]+m.z22[0]+m.lack2[0] == 100) 
  m.limit.add(expr= m.z12[1]+m.z22[1]+m.lack2[1] == 112.5) 

  SolverFactory('glpk', executable='/usr/bin/glpsol').solve(m)
  return m


In [0]:
model2 = model2()

In [75]:
m2 = pd.DataFrame(columns=['x1', 'x2', 'objective'], index=['value']) 
m2['x1'] = model2.x1()
m2['x2'] = model2.x2()
m2['objective'] = -model2.obj()
m2

,x1,x2,objective
value,125.0,100.0,449.75
